In [0]:
Extraction-based summarization Суммирование на основе извлечения
Извлечение ключевых фраз из исходного документа и их объединение в резюме. Производится в соответствии с заданной метрикой без каких-либо изменений в текстах.

Abstraction-based summarization Обобщение на основе абстракций
перефразирование и сокращение частей исходного документа, обобщения текста, преодолевает грамматические несоответствия экстрактивного метода. Обобщения абстрактного текста создают новые фразы и предложения, передающие наиболее полезную информацию из исходного текста.Абстракция работает лучше, чем извлечение. Однако алгоритмы суммирования текста, необходимые для абстракции, сложнее разработать; 

ROUGE-1 относится к перекрытию униграмм (каждого слова) между системой и справочными данными.
ROUGE-2 относится к перекрытию биграмм между системой и справочными сводками.
ROUGE-L: статистика на основе самой длинной общей подпоследовательности (LCS) 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#ssh

In [0]:
#create ssh connect

import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
#Print root password
print("Root password: {}".format(password))
#Get public address
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

#Requirements

In [0]:
import requests
import os
import random
import tarfile
import pandas as pd
import zipfile
import multiprocess
import json
from glob import glob

In [3]:
!git clone https://github.com/brs1977/PreSumm.git

!pip install pytorch_transformers
# !pip install torch
!pip install pytorch-pretrained-bert
!pip install tensorboardX

# !pip install pyrouge
#pyrouge
!sudo apt-get install libxml-parser-perl 

!git clone https://github.com/andersjo/pyrouge.git rouge
%cd /content/rouge/tools/ROUGE-1.5.5/data
!rm WordNet-2.0.exc.db
!./WordNet-2.0-Exceptions/buildExeptionDB.pl ./WordNet-2.0-Exceptions ./smart_common_words.txt ./WordNet-2.0.exc.db

%cd /content
!git clone https://github.com/bheinzerling/pyrouge
%cd /content/pyrouge
!python setup.py install
!pyrouge_set_rouge_path '/content/rouge/tools/ROUGE-1.5.5'
!python -m pyrouge.test


Cloning into 'PreSumm'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 529 (delta 110), reused 62 (delta 35), pack-reused 374
Receiving objects: 100% (529/529), 13.03 MiB | 5.46 MiB/s, done.
Resolving deltas: 100% (359/359), done.
     |████████████████████████████████| 184kB 35.7MB/s 
     |████████████████████████████████| 1.0MB 41.5MB/s 
     |████████████████████████████████| 870kB 69.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=0483e455ce4e25f87df6f41180ea2f5a68d15d3c5970194807944f9dccbc7377
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
     |████████████████████████████████| 133kB 18.5MB/s 
     |████████████████████████████████| 204kB 20.9MB/s 
Reading package lists... Done
Building dependency tree       
Reading state informa

##Download Stanford CoreNLP

In [0]:
%cd /content
#########
!wget http://nlp.stanford.edu/software/stanford-corenlp-full-2017-06-09.zip
!unzip stanford-corenlp-full-2017-06-09.zip
%env CLASSPATH=/content/stanford-corenlp-full-2017-06-09/stanford-corenlp-3.8.0.jar


/content
--2020-01-20 19:17:47--  http://nlp.stanford.edu/software/stanford-corenlp-full-2017-06-09.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/software/stanford-corenlp-full-2017-06-09.zip [following]
--2020-01-20 19:17:48--  https://nlp.stanford.edu/software/stanford-corenlp-full-2017-06-09.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 390211140 (372M) [application/zip]
Saving to: ‘stanford-corenlp-full-2017-06-09.zip’

stanford-corenlp-fu 100%[===================>] 372.13M  12.1MB/s    in 34s     

2020-01-20 19:18:22 (11.0 MB/s) - ‘stanford-corenlp-full-2017-06-09.zip’ saved [390211140/390211140]

Archive:  stanford-corenlp-full-2017-06-09.zip
   creating: stanford-corenlp-full-2017-06-09/
  inflating

In [0]:
# %env CLASSPATH=/content/stanford-corenlp-full-2017-06-09/stanford-corenlp-3.8.0.jar
!echo $CLASSPATH

/content/stanford-corenlp-full-2017-06-09/stanford-corenlp-3.8.0.jar


##Kaggle env

In [4]:
#!pip install -q kaggle

#kaggle key
!mkdir ~/.kaggle
!cp /content/drive/My\ Drive/kaggle.json ~/.kaggle
!ls ~/.kaggle

kaggle.json


##Load kaggle data

In [5]:
%cd /content
!kaggle competitions download -c title-generation

/content
 78% 5.00M/6.39M [00:00<00:00, 14.6MB/s]
100% 6.39M/6.39M [00:00<00:00, 18.4MB/s]
  0% 0.00/778k [00:00<?, ?B/s]
100% 778k/778k [00:00<00:00, 105MB/s]
 94% 41.0M/43.7M [00:01<00:00, 20.3MB/s]
100% 43.7M/43.7M [00:01<00:00, 32.3MB/s]
  0% 0.00/905k [00:00<?, ?B/s]
100% 905k/905k [00:00<00:00, 59.7MB/s]


In [6]:
!mkdir data
!unzip sample_submission.csv.zip -d data
!unzip vocs.pkl.zip -d data
!unzip train.csv.zip -d data
!mv test.csv data

!ls data

Archive:  sample_submission.csv.zip
  inflating: data/sample_submission.csv  
Archive:  vocs.pkl.zip
  inflating: data/vocs.pkl           
Archive:  train.csv.zip
  inflating: data/train.csv          
sample_submission.csv  test.csv  train.csv  vocs.pkl


#Data prepare

In [0]:
import pandas as pd

train = pd.read_csv('/content/data/train.csv', encoding='utf8')
test = pd.read_csv('/content/data/test.csv', encoding='utf8')

In [0]:
test.head

###Splitting and Tokenization

In [0]:
import numpy as np
import os

#It is processed this way because this repo was based on the ArXiv dataset, which
# has the data in [abstract] @highlight [title] format
arxiv_dir = '/content/PreSumm/arxiv_raw'
tokenized_dir = '/content/merged_arxiv_tokenized'

%cd /content

!rm $arxiv_dir -r
!mkdir $arxiv_dir
!rm $tokenized_dir -r
!mkdir $tokenized_dir

#split val test
rand_split = [.1]
for row in train.iterrows():
  i, (abstract,title) = row
  sampletext =  abstract + '\n\n@highlight\n\n' + title

  r = np.random.rand()
  #  split val train
  fn = 'valid' if r < rand_split[0] else 'train'  

  fn = f'{fn}.{i}.story'
  fn = os.path.join(arxiv_dir, fn)
  with open(fn, "w") as text_file:
    text_file.write(sampletext)

for row in test.iterrows():
  i, abstract = row
  sampletext =  abstract[0] + '\n'
  fn = f'test.{i}.story'
  fn = os.path.join(arxiv_dir, fn)
  with open(fn, "w") as text_file:
    text_file.write(sampletext)



/content
rm: cannot remove '/content/PreSumm/arxiv_raw': No such file or directory
rm: cannot remove '/content/merged_arxiv_tokenized': No such file or directory


In [0]:
%%capture
!mkdir /content/logs
%cd /content/PreSumm/src
!python preprocess.py -mode tokenize -raw_path $arxiv_dir -save_path $tokenized_dir

###Format to Simpler Json Files

In [0]:
%%capture
!python preprocess.py -mode format_to_lines -raw_path $tokenized_dir -save_path ../json_data/arxiv -shard_size 20000 -n_cpus 1 

###Format to PyTorch Files

In [0]:
#%%capture
%cd /content/PreSumm/src
!python preprocess.py -mode format_to_bert -raw_path ../json_data/ -save_path ../bert_data/ -lower -n_cpus 1 -log_file ../logs/preprocess.log

/content/PreSumm/src
[('train', '../json_data/cnndm_sample.train.0.json', Namespace(dataset='', log_file='../logs/preprocess.log', lower=True, map_path='../../data/', max_src_nsents=100, max_src_ntokens_per_sent=200, max_tgt_ntokens=500, min_src_nsents=3, min_src_ntokens_per_sent=5, min_tgt_ntokens=5, mode='format_to_bert', n_cpus=1, pretrained_model='bert', raw_path='../json_data/', save_path='../bert_data/', select_mode='greedy', shard_size=2000, use_bert_basic_tokenizer=False, vocab_file='bert-base-uncased'), '../bert_data/cnndm_sample.train.0.bert.pt'), ('train', '../json_data/arxiv.train.6.json', Namespace(dataset='', log_file='../logs/preprocess.log', lower=True, map_path='../../data/', max_src_nsents=100, max_src_ntokens_per_sent=200, max_tgt_ntokens=500, min_src_nsents=3, min_src_ntokens_per_sent=5, min_tgt_ntokens=5, mode='format_to_bert', n_cpus=1, pretrained_model='bert', raw_path='../json_data/', save_path='../bert_data/', select_mode='greedy', shard_size=2000, use_bert_bas

In [0]:
#Save pretraining data
%cd ../bert_data
!zip pre_sum_data.zip *.pt
#!rm /content/drive/My\ Drive/nlp/bert_sum_data.zip
!cp pre_sum_data.zip /content/drive/My\ Drive/nlp

In [0]:
# import torch 
# nd = torch.load('../bert_data/arxiv.test.0.bert.pt')
# len(nd)
# l = glob.glob(tokenized_dir+'/*.json')
# len(l)
# import json
# jd = json.load(open('/content/BertSum/json_data/arxiv.new.0.json'))
# len(jd)

1000

#Train

In [22]:
!git pull origin master 

remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 6 (delta 5), reused 6 (delta 5), pack-reused 0
Unpacking objects: 100% (6/6), done.
From https://github.com/brs1977/PreSumm
 * branch            master     -> FETCH_HEAD
   81217e9..fc9c431  master     -> origin/master
Updating 81217e9..fc9c431
Fast-forward
 src/models/predictor.py  | 3 +++
 src/train_abstractive.py | 2 +-
 2 files changed, 4 insertions(+), 1 deletion(-)


In [7]:
#unzip prepared data
!unzip /content/drive/My\ Drive/nlp/pre_sum_data.zip -d /content/PreSumm/bert_data/


Archive:  /content/drive/My Drive/nlp/pre_sum_data.zip
  inflating: /content/PreSumm/bert_data/arxiv.test.0.bert.pt  
  inflating: /content/PreSumm/bert_data/arxiv.train.0.bert.pt  
  inflating: /content/PreSumm/bert_data/arxiv.train.1.bert.pt  
  inflating: /content/PreSumm/bert_data/arxiv.train.2.bert.pt  
  inflating: /content/PreSumm/bert_data/arxiv.train.3.bert.pt  
  inflating: /content/PreSumm/bert_data/arxiv.train.4.bert.pt  
  inflating: /content/PreSumm/bert_data/arxiv.train.5.bert.pt  
  inflating: /content/PreSumm/bert_data/arxiv.train.6.bert.pt  
  inflating: /content/PreSumm/bert_data/arxiv.valid.0.bert.pt  
  inflating: /content/PreSumm/bert_data/cnndm_sample.train.0.bert.pt  


In [14]:
3600*4

14400

In [15]:
%cd /content/PreSumm/src
bert_data_path = '../bert_data/arxiv'
model_path = '/content/drive/My\ Drive/nlp/nlp_model/pre_summ'
train_from = '/content/drive/My\ Drive/nlp/model_step_3600.pt' # '/content/drive/My\ Drive/nlp/nlp_model/pre_summ/model_step_7700.pt'
#train

!python train.py  -task abs -mode train -log_file /content/PreSumm/logs/abs_bert_arxiv  \
  -bert_data_path $bert_data_path -model_path $model_path\
  -accum_count 5 -save_checkpoint_steps 3600 -batch_size 512 -use_interval true \
  -train_steps 14500 -report_every 50 -use_bert_emb true \
  -lr 2e-3 -decay_method noam -warmup_steps 100 \
  -dec_dropout 0.2  -max_pos 512 -visible_gpus 0 

# # -sep_optim true -decay_method noam -lr_bert 1e-6 -lr_dec 1e-6 -warmup_steps_bert 20000 -warmup_steps_dec 10000 
# !python train.py  -task abs -mode train -log_file /content/PreSumm/logs/abs_bert_arxiv  \
#   -bert_data_path $bert_data_path -model_path $model_path\
#   -accum_count 5 -save_checkpoint_steps 3600 -batch_size 512 -use_interval true \
#   -train_steps 14500 -report_every 50 -use_bert_emb true \
#   -sep_optim true -decay_method noam -lr_bert 2e-6 -lr_dec 2e-6 -warmup_steps_bert 20000 -warmup_steps_dec 10000 \
#   -dec_dropout 0.2  -max_pos 512 -visible_gpus 0 


#resume train
#было -lr_bert 0.002
# !python train.py  -task abs -mode train -log_file /content/PreSumm/logs/abs_bert_arxiv  \
#   -bert_data_path $bert_data_path -train_from $train_from -model_path $model_path\
#   -accum_count 5 -save_checkpoint_steps 3600 -batch_size 512 -use_interval true \
#   -train_steps 14500 -report_every 100 -use_bert_emb true \
#   -sep_optim true -decay_method None -lr_bert 1e-6 -lr_dec 0.2 -warmup_steps_bert 20000 -warmup_steps_dec 10000 \
#   -dec_dropout 0.2  -max_pos 512 -visible_gpus 0 
  
# !python train.py  -task abs -mode train -bert_data_path $bert_data_path -dec_dropout 0.2 \
#    -model_path $model_path -sep_optim true -lr_bert 0.002 -lr_dec 0.2 -save_checkpoint_steps 3600 \
#    -batch_size 512 -train_steps 14500 -report_every 1000 -accum_count 5 -use_bert_emb true \
#    -use_interval true -warmup_steps_bert 20000 -warmup_steps_dec 10000 -max_pos 512 -visible_gpus 0 \
#    -log_file /content/PreSumm/logs/abs_bert_arxiv  

#calculate -max_length 70 -alpha 0.95 -min_length 10



/content/PreSumm/src
[2020-01-23 06:11:28,764 INFO] Namespace(accum_count=5, alpha=0.6, batch_size=512, beam_size=5, bert_data_path='../bert_data/arxiv', bert_model='bert-base-uncased', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, decay_method='noam', enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], label_smoothing=0.1, large=False, load_from_extractive='', log_file='/content/PreSumm/logs/abs_bert_arxiv', lr=0.002, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=150, max_pos=512, max_tgt_len=140, min_length=15, mode='train', model_path='/content/drive/My Drive/nlp/nlp_model/pre_summ', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=50, report_rouge=True, result_path='../results/cnndm',

In [18]:
0.00000283
2e-6 == 0.000002 

Namespace(accum_count=1, alpha=0.95, batch_size=140, beam_size=5, bert_data_path='../bert_data/arxiv', 
          
          bert_model='bert-base-uncased', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, 

          dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512,           
          enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, 
          ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], 
          label_smoothing=0.1, large=False, load_from_extractive='', log_file='/content/PreSumm/logs/log_val_abs_arxiv', 

          lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=200, max_pos=512, max_tgt_len=140, 
          min_length=50, mode='validate', model_path='/content/drive/My Drive/nlp/nlp_model/pre_summ', 
          optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, 
          report_rouge=True, result_path='/content/PreSumm/logs/abs_arxiv', save_checkpoint_steps=5, 
          seed=666, sep_optim=True, share_emb=False, task='abs', temp_dir='../temp', test_all=False, 

          test_batch_size=200, test_from='/content/drive/My Drive/nlp/nlp_model/pre_summ/model_step_7200.pt',           
          test_start_from=-1, train_from='', train_steps=1000, use_bert_emb=False, use_interval=True, 
          
          visible_gpus='0', warmup_steps=8000, warmup_steps_bert=8000, warmup_steps_dec=8000, world_size=1)

True

#Validate

In [21]:
%cd /content/PreSumm/src
bert_data_path = '../bert_data/arxiv'
test_from = '/content/drive/My\ Drive/nlp/nlp_model/pre_summ/model_step_7200.pt'
model_path = '/content/drive/My\ Drive/nlp/nlp_model/pre_summ'

#val
!python train.py -task abs -mode validate -test_from $test_from -bert_data_path $bert_data_path -gpu_ranks 0 -visible_gpus 0 -report_rouge true \
  -bert_data_path ../bert_data/arxiv -log_file /content/PreSumm/logs/log_val_abs_arxiv -sep_optim true -use_interval true -max_pos 512 \
  -max_length 70 -alpha 0.95 -min_length 10 -result_path /content/PreSumm/logs/abs_arxiv \
  -test_batch_size 512 -model_path $model_path


/content/PreSumm/src
[2020-01-22 09:51:44,449 INFO] Loading checkpoint from /content/drive/My Drive/nlp/nlp_model/pre_summ/model_step_7200.pt
Namespace(accum_count=1, alpha=0.95, batch_size=140, beam_size=5, bert_data_path='../bert_data/arxiv', bert_model='bert-base-uncased', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], label_smoothing=0.1, large=False, load_from_extractive='', log_file='/content/PreSumm/logs/log_val_abs_arxiv', lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=200, max_pos=512, max_tgt_len=140, min_length=50, mode='validate', model_path='/content/drive/My Drive/nlp/nlp_model/pre_summ', optim='adam', param_init=0, param_init_glorot=True, recall_eval=Fa

#Predict

In [29]:
%cd /content/PreSumm/src
bert_data_path = '../bert_data/arxiv'
test_from = '/content/drive/My\ Drive/nlp/nlp_model/pre_summ/model_step_7200.pt' #'/content/drive/My\ Drive/nlp/model_step_3600.pt' # 
model_path = '/content/drive/My\ Drive/nlp/nlp_model/pre_summ'
#val
!python train.py -task abs -mode predict -test_from $test_from -bert_data_path $bert_data_path -gpu_ranks 0 -visible_gpus 0 -report_rouge true \
  -bert_data_path ../bert_data/arxiv -log_file /content/PreSumm/logs/log_val_abs_arxiv -sep_optim true -use_interval true -max_pos 512 \
  -max_length 70 -alpha 0.95 -min_length 10 -result_path /content/PreSumm/logs/abs_arxiv \
  -test_batch_size 512 -model_path $model_path

#max_length (int): maximum length output to produce


/content/PreSumm/src
[2020-01-22 12:22:11,717 INFO] Loading checkpoint from /content/drive/My Drive/nlp/nlp_model/pre_summ/model_step_7200.pt
Namespace(accum_count=1, alpha=0.95, batch_size=140, beam_size=5, bert_data_path='../bert_data/arxiv', bert_model='bert-base-uncased', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], label_smoothing=0.1, large=False, load_from_extractive='', log_file='/content/PreSumm/logs/log_val_abs_arxiv', lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=200, max_pos=512, max_tgt_len=140, min_length=50, mode='predict', model_path='/content/drive/My Drive/nlp/nlp_model/pre_summ', optim='adam', param_init=0, param_init_glorot=True, recall_eval=Fal

In [30]:
!ls /content/PreSumm/logs #/abs_arxiv.2200.candidate
import pandas as pd 
# src = pd.read_csv('/content/PreSumm/logs/abs_arxiv.2200.raw_src', nrows=1)
pred = pd.read_csv('/content/PreSumm/logs/abs_arxiv.-1.candidate', header=None, sep=';')
test = pd.read_csv('/content/data/test.csv', encoding='utf8')

abs_arxiv.-1.candidate	abs_arxiv.7200.candidate  abs_bert_arxiv
abs_arxiv.-1.gold	abs_arxiv.7200.gold	  log_val_abs_arxiv
abs_arxiv.-1.raw_src	abs_arxiv.7200.raw_src


In [31]:
pred.iloc[:,0]

# 0      on planar embedding theorem for planar embeddi...
# 1      hodge conjectures and hodge conjectures in the...
# 2      discrete logarithms and discrete logarithms in...
# 3      nonmonotonotonic and semantical rules for nonm...
# 4      on the theory of the interaction between the a...

0      a new proof of planar embedding theorem in pla...
1      on the number of the hodge conjecture of the n...
2      on the discrete logarithms in the polynomial l...
3      nonmonotonic and semantical rules for nonmonot...
4      a note on the theory of the theory and dr and ...
                             ...                        
995    on the plane of the projective varieties of su...
996    on the rate of non-square complex orthogonal d...
997    a multiwavelength study of the weak cluster r ...
998    on the relationship of the number of the quijo...
999    fast and diversity and diversity in the number...
Name: 0, Length: 1000, dtype: object

In [0]:
submission_df = pd.DataFrame({'abstract': test.iloc[:,0], 'title': pred.iloc[:,0]})
submission_df.to_csv('/content/predicted_titles.csv', index=False)


In [33]:
import string
from nltk.util import ngrams
import numpy as np
import pandas as pd
import pickle

def generate_csv(input_file='/content/predicted_titles.csv',
                 output_file='submission.csv',
                 voc_file='/content/data/vocs.pkl'):
    '''
    Generates file in format required for submitting result to Kaggle
    
    Parameters:
        input_file (str) : path to csv file with your predicted titles.
                           Should have two fields: abstract and title
        output_file (str) : path to output submission file
        voc_file (str) : path to voc.pkl file
    '''
    data = pd.read_csv(input_file)
    with open(voc_file, 'rb') as voc_file:
        vocs = pickle.load(voc_file)

    with open(output_file, 'w') as res_file:
        res_file.write('Id,Predict\n')
        
    output_idx = 0
    for row_idx, row in data.iterrows():
        trg = row['title']
        trg = trg.split('<q>')[0]
        trg = trg.translate(str.maketrans('', '', string.punctuation)).lower().split()
        trg.extend(['_'.join(ngram) for ngram in list(ngrams(trg, 2)) + list(ngrams(trg, 3))])
        
        VOCAB_stoi = vocs[row_idx]
        trg_intersection = set(VOCAB_stoi.keys()).intersection(set(trg))
        trg_vec = np.zeros(len(VOCAB_stoi))    

        for word in trg_intersection:
            trg_vec[VOCAB_stoi[word]] = 1

        with open(output_file, 'a') as res_file:
            for is_word in trg_vec:
                res_file.write('{0},{1}\n'.format(output_idx, int(is_word)))
                output_idx += 1


generate_csv()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: generator 'ngrams' raised StopIteration


In [34]:
#2000 - 0.12958
# [2020-01-21 08:44:06,631 INFO] Validation perplexity: 30.1915
# [2020-01-21 08:44:06,631 INFO] Validation accuracy: 41.727
# !kaggle competitions submit -c title-generation -m "PreSumm test model_step_2000.pt" -f submission.csv 

!kaggle competitions submit -c title-generation -m "PreSumm test model_step_3600.pt" -f submission.csv 

100% 2.84M/2.84M [00:08<00:00, 349kB/s]
Successfully submitted to Arxiv Title Generation

In [0]:
import torch 
dt = torch.load('../bert_data/arxiv.valid.0.bert.pt')
dt[1]
# %cd ../bert_data
# !zip bert_sum_data.zip *.pt
#!cp bert_sum_data.zip /content/drive/My\ Drive/nlp/bert_sum_data1.zip

{'clss': [0, 44, 83, 110, 151, 192, 246, 276, 311, 336],
 'segs': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [0]:
%cd /content/PreSumm/src/

bert_data_path = '../bert_data/arxiv'
model_path = '/content/drive/My\ Drive/nlp/nlp_model/arxiv_bert/models' #'../logs/model' #'/content/drive/My\ Drive/nlp/nlp_model/bertsum'
bert_config_path = '/content/scibert_scivocab_uncased/bert_config.json'
bert_model = '/content/scibert_scivocab_uncased/weights.tar.gz'
log_file = '/content/PreSumm/logs/abs_bert_arxiv'
train_from = '/content/drive/My\ Drive/nlp/nlp_model/arxiv_bert/models/model_step_121000.pt'

#start train
# !python train.py  -mode train -encoder transformer -bert_data_path $bert_data_path -dropout 0.2 \
#    -bert_model $bert_model \
#    -model_path $model_path -lr 2e-3 -save_checkpoint_steps 500 \
#    -batch_size 512 -train_steps 10000 -report_every 500 -accum_count 5 \
#    -use_interval true -warmup_steps 20000 -visible_gpus 0 \
#    -log_file $log_file

#resume train
!python train.py -mode train -encoder transformer -bert_data_path $bert_data_path -dropout 0.2 \
  -train_from $train_from -model_path $model_path -bert_config_path $bert_config_path -bert_model $bert_model \
  -lr 5e-2 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 500 -save_checkpoint_steps 11000 \
  -batch_size 512 -decay_method noam -train_steps 200000 -accum_count 7 -log_file $log_file \
  -use_interval true -warmup_steps 100 


#resume train
# !python train.py -task abs -mode train -train_from ../models/model_step_10000.pt \
#   -bert_data_path ../bert_data/arxiv -dec_dropout 0.2  -model_path ../models \
#   -sep_optim true -lr_bert 0.002 -lr_dec 0.2 -save_checkpoint_steps 1000 -batch_size 512 \
#   -train_steps 20000 -report_every 1000 -accum_count 5 -use_bert_emb true -use_interval true \
#   -warmup_steps_bert 20000 -warmup_steps_dec 20000 -max_pos 512 -visible_gpus 0 -log_file /content/logs/abs_bert_arxiv  



# !python train.py -task abs -mode train -train_from ../models/model_step_10000.pt \
#   -bert_data_path ../bert_data/arxiv -dec_dropout 0.2  -model_path ../models \
#   -sep_optim true -lr_bert 0.002 -lr_dec 0.2 -save_checkpoint_steps 1000 -batch_size 512 \
#   -train_steps 20000 -report_every 1000 -accum_count 5 -use_bert_emb true -use_interval true \
#   -warmup_steps_bert 20000 -warmup_steps_dec 20000 -max_pos 512 -visible_gpus 0 -log_file /content/logs/abs_bert_arxiv  




/content/BertSum/src
[2020-01-20 07:34:52,898 INFO] Device ID 0
[2020-01-20 07:34:52,898 INFO] Device cuda
[2020-01-20 07:34:52,940 INFO] loading archive file /content/scibert_scivocab_uncased/weights.tar.gz
[2020-01-20 07:34:52,940 INFO] extracting archive file /content/scibert_scivocab_uncased/weights.tar.gz to temp dir /tmp/tmpk36v_qhh
[2020-01-20 07:34:56,413 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 31090
}

[2020-01-20 07:35:07,701 INFO] Loading checkpoint from /content/drive/My Drive/nlp/nlp_model/arxiv_bert/models/model_step_121000.pt
[2020-01-20 07:35:17,047 INFO] Summarizer(
  (bert): Bert(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(31090, 768, p

#Validate

In [0]:
%cd /content/BertSum/src/

bert_data_path = '../bert_data/arxiv'
model_path = '/content/drive/My\ Drive/nlp/nlp_model/arxiv_bert/models' #'../logs/model' #'/content/drive/My\ Drive/nlp/nlp_model/bertsum'
bert_config_path = '/content/scibert_scivocab_uncased/bert_config.json'
bert_model = '/content/scibert_scivocab_uncased/weights.tar.gz'
log_file = '/content/BertSum/logs/abs_bert_arxiv'
train_from = '/content/drive/My\ Drive/nlp/nlp_model/arxiv_bert/models/model_step_110000.pt'

!python train.py -mode validate -encoder transformer -dropout 0.2 \
  -bert_data_path $bert_data_path -test_from $train_from -model_path $model_path -bert_config_path $bert_config_path  \
  -bert_model $bert_model -temp_dir /tmp \
  -lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 500 -save_checkpoint_steps 15000 \
  -batch_size 512 -decay_method noam -train_steps 100000 -accum_count 5 -log_file $log_file \
  -use_interval true -warmup_steps 100 

# 30000
# [2020-01-18 06:21:12,407 INFO] Rouges at step 30000 
# >> ROUGE-F(1/2/3/l): 18.44/8.08/16.21
# ROUGE-R(1/2/3/l): 70.82/33.27/62.73  
# 60000
# [2020-01-18 09:46:49,751 INFO] Rouges at step 60000 
# >> ROUGE-F(1/2/3/l): 18.53/8.17/16.30
# ROUGE-R(1/2/3/l): 71.05/33.57/62.99

# 75000
# [2020-01-18 11:35:12,508 INFO] Rouges at step 75000 
# >> ROUGE-F(1/2/3/l): 18.57/8.19/16.33
# ROUGE-R(1/2/3/l): 71.17/33.67/63.12

# 88000
# [2020-01-19 08:17:26,555 INFO] Rouges at step 88000 
# >> ROUGE-F(1/2/3/l): 18.58/8.20/16.35
# ROUGE-R(1/2/3/l): 71.22/33.70/63.17

# 99000
# [2020-01-19 09:50:21,531 INFO] Rouges at step 99000 
# >> ROUGE-F(1/2/3/l): 18.57/8.20/16.34
# ROUGE-R(1/2/3/l): 71.18/33.68/63.13


# 110000
# [2020-01-19 06:41:30,907 INFO] Rouges at step 110000 
# >> ROUGE-F(1/2/3/l): 18.47/8.10/16.23
# ROUGE-R(1/2/3/l): 70.90/33.35/62.82


# 121000
# [2020-01-19 06:06:43,396 INFO] Rouges at step 121000 
# >> ROUGE-F(1/2/3/l): 18.48/8.09/16.23
# ROUGE-R(1/2/3/l): 70.92/33.28/62.79

# 121000 lr 5e-2
# [2020-01-19 16:25:25,110 INFO] Rouges at step 121000 
# >> ROUGE-F(1/2/3/l): 18.57/8.20/16.34
# ROUGE-R(1/2/3/l): 71.17/33.69/63.13

# 132000
# [2020-01-20 10:52:15,305 INFO] Rouges at step 132000 
# >> ROUGE-F(1/2/3/l): 18.57/8.20/16.34
# ROUGE-R(1/2/3/l): 71.19/33.70/63.15

#143000
# [2020-01-20 10:36:06,954 INFO] Rouges at step 143000 
# >> ROUGE-F(1/2/3/l): 18.53/8.17/16.30
# ROUGE-R(1/2/3/l): 71.11/33.63/63.07


/content/BertSum/src
[2020-01-19 06:30:23,097 INFO] Loading checkpoint from /content/drive/My Drive/nlp/nlp_model/arxiv_bert/models/model_step_110000.pt
Namespace(accum_count=5, batch_size=512, bert_config_path='/content/scibert_scivocab_uncased/bert_config.json', bert_data_path='../bert_data/arxiv', bert_model='/content/scibert_scivocab_uncased/weights.tar.gz', beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='noam', dropout=0.2, encoder='transformer', ff_size=512, gpu_ranks=[0], heads=4, hidden_size=128, inter_layers=2, log_file='/content/BertSum/logs/abs_bert_arxiv', lr=0.002, max_grad_norm=0, mode='validate', model_path='/content/drive/My Drive/nlp/nlp_model/arxiv_bert/models', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=500, report_rouge=True, result_path='../results/cnndm', rnn_size=512, save_checkpoint_steps=15000, seed=666, temp_dir='/tmp', test_all=False, test_from='/content/drive/My Drive/nlp/nlp_model/arxiv_bert/mod

In [0]:
# Namespace(accum_count=5, batch_size=512, bert_config_path='/content/scibert_scivocab_uncased/bert_config.json', 
#           bert_data_path='../bert_data/arxiv', bert_model='/content/scibert_scivocab_uncased/weights.tar.gz', 
#           beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='noam', dropout=0.2, 
#           encoder='transformer', ff_size=512, gpu_ranks=[0], heads=4, hidden_size=128, inter_layers=2, 
#           log_file='/content/BertSum/logs/abs_bert_arxiv', lr=0.002, max_grad_norm=0, mode='validate', 
#           model_path='/content/drive/My Drive/nlp/nlp_model/arxiv_bert/models', optim='adam', param_init=0, 
#           param_init_glorot=True, recall_eval=False, report_every=500, report_rouge=True, 
#           result_path='../results/cnndm', rnn_size=512, save_checkpoint_steps=15000, seed=666, 
#           temp_dir='../temp', test_all=False, test_from='', 
#           train_from='/content/drive/My Drive/nlp/nlp_model/arxiv_bert/models/model_step_30000.pt', 
#           train_steps=100000, use_interval=True, visible_gpus='0', warmup_steps=100, world_size=1)


#Test

In [0]:

%cd /content/BertSum/src/

bert_data_path = '../bert_data/arxiv'
model_path = '/content/drive/My\ Drive/nlp/nlp_model/arxiv_bert/models' #'../logs/model' #'/content/drive/My\ Drive/nlp/nlp_model/bertsum'
bert_config_path = '/content/scibert_scivocab_uncased/bert_config.json'
bert_model = '/content/scibert_scivocab_uncased/weights.tar.gz'
log_file = '/content/BertSum/logs/abs_bert_arxiv'
test_from = '/content/drive/My\ Drive/nlp/nlp_model/arxiv_bert/models/model_step_132000.pt'

!python train.py -mode test -encoder transformer -dropout 0.2 \
  -bert_data_path $bert_data_path -test_from $test_from -model_path $model_path -bert_config_path $bert_config_path  \
  -bert_model $bert_model -temp_dir /tmp \
  -lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 500 -save_checkpoint_steps 15000 \
  -batch_size 512 -decay_method noam -train_steps 100000 -accum_count 5 -log_file $log_file \
  -use_interval true -warmup_steps 100 

/content/BertSum/src
[2020-01-20 10:44:13,603 INFO] Loading checkpoint from /content/drive/My Drive/nlp/nlp_model/arxiv_bert/models/model_step_132000.pt
Namespace(accum_count=5, batch_size=512, bert_config_path='/content/scibert_scivocab_uncased/bert_config.json', bert_data_path='../bert_data/arxiv', bert_model='/content/scibert_scivocab_uncased/weights.tar.gz', beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='noam', dropout=0.2, encoder='transformer', ff_size=512, gpu_ranks=[0], heads=4, hidden_size=128, inter_layers=2, log_file='/content/BertSum/logs/abs_bert_arxiv', lr=0.002, max_grad_norm=0, mode='test', model_path='/content/drive/My Drive/nlp/nlp_model/arxiv_bert/models', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=500, report_rouge=True, result_path='../results/cnndm', rnn_size=512, save_checkpoint_steps=15000, seed=666, temp_dir='/tmp', test_all=False, test_from='/content/drive/My Drive/nlp/nlp_model/arxiv_bert/models/

#Predict

In [0]:
!git pull origin master

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/brs1977/BertSum
 * branch            master     -> FETCH_HEAD
   a46e57c..7eb88bb  master     -> origin/master
Updating a46e57c..7eb88bb
Fast-forward
 src/models/trainer.py | 8 ++++----
 1 file changed, 4 insertions(+), 4 deletions(-)


In [0]:
%cd /content/BertSum/src/

bert_data_path = '../bert_data/arxiv'
model_path = '/content/drive/My\ Drive/nlp/nlp_model/arxiv_bert/models' #'../logs/model' #'/content/drive/My\ Drive/nlp/nlp_model/bertsum'
bert_config_path = '/content/scibert_scivocab_uncased/bert_config.json'
bert_model = '/content/scibert_scivocab_uncased/weights.tar.gz'
log_file = '/content/BertSum/logs/abs_bert_arxiv'
test_from = '/content/drive/My\ Drive/nlp/nlp_model/arxiv_bert/model_step_88000.pt'
result_file = '/content/predicted_titles.csv'

!python train.py -mode predict -encoder transformer -dropout 0.2 \
  -bert_data_path $bert_data_path -test_from $test_from -model_path $model_path -bert_config_path $bert_config_path  \
  -result_file $result_file -bert_model $bert_model -temp_dir /tmp \
  -lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 500 -save_checkpoint_steps 15000 \
  -batch_size 512 -decay_method noam -train_steps 100000 -accum_count 5 -log_file $log_file \
  -use_interval true -warmup_steps 100 

/content/BertSum/src
[2020-01-20 17:35:56,149 INFO] Loading checkpoint from /content/drive/My Drive/nlp/nlp_model/arxiv_bert/model_step_88000.pt
Namespace(accum_count=5, batch_size=512, bert_config_path='/content/scibert_scivocab_uncased/bert_config.json', bert_data_path='../bert_data/arxiv', bert_model='/content/scibert_scivocab_uncased/weights.tar.gz', beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='noam', dropout=0.2, encoder='transformer', ff_size=512, gpu_ranks=[0], heads=4, hidden_size=128, inter_layers=2, log_file='/content/BertSum/logs/abs_bert_arxiv', lr=0.002, max_grad_norm=0, mode='predict', model_path='/content/drive/My Drive/nlp/nlp_model/arxiv_bert/models', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=500, report_rouge=True, result_file='/content/predicted_titles.csv', result_path='../results/cnndm', rnn_size=512, save_checkpoint_steps=15000, seed=666, temp_dir='/tmp', test_all=False, test_from='/content/drive/M

In [0]:
# import pandas as pd
# pt = pd.read_csv('/content/predicted_titles.csv')
# pt.head()

,abstract,title
0,we give existence results for solutions of the...,we give existence results for solutions of the...
1,"in this article , i will prove a recurrence th...","in this article , i will prove a recurrence th..."
2,we show both theoretically and experimentally ...,we show both theoretically and experimentally ...
3,we summarize here the first results obtained u...,we summarize here the first results obtained u...
4,we study a 1d transport equation with nonlocal...,we study a 1d transport equation with nonlocal...


In [0]:
pt.title[0].split('<q>')[0]
# import torch
# nd = torch.load('../bert_data/arxiv.new.0.bert.pt')
# len(nd)

'we give existence results for solutions of the prescribed scalar curvature equation on $ s ^ 3 $ , when the curvature function is a positive morse function and satisfies an index-count condition .'

In [0]:
import string
from nltk.util import ngrams
import numpy as np
import pandas as pd
import pickle

def generate_csv(input_file='/content/predicted_titles.csv',
                 output_file='submission.csv',
                 voc_file='/content/data/vocs.pkl'):
    '''
    Generates file in format required for submitting result to Kaggle
    
    Parameters:
        input_file (str) : path to csv file with your predicted titles.
                           Should have two fields: abstract and title
        output_file (str) : path to output submission file
        voc_file (str) : path to voc.pkl file
    '''
    data = pd.read_csv(input_file)
    with open(voc_file, 'rb') as voc_file:
        vocs = pickle.load(voc_file)

    with open(output_file, 'w') as res_file:
        res_file.write('Id,Predict\n')
        
    output_idx = 0
    for row_idx, row in data.iterrows():
        trg = row['title']
        trg = trg.split('<q>')[0]
        trg = trg.translate(str.maketrans('', '', string.punctuation)).lower().split()
        trg.extend(['_'.join(ngram) for ngram in list(ngrams(trg, 2)) + list(ngrams(trg, 3))])
        
        VOCAB_stoi = vocs[row_idx]
        trg_intersection = set(VOCAB_stoi.keys()).intersection(set(trg))
        trg_vec = np.zeros(len(VOCAB_stoi))    

        for word in trg_intersection:
            trg_vec[VOCAB_stoi[word]] = 1

        with open(output_file, 'a') as res_file:
            for is_word in trg_vec:
                res_file.write('{0},{1}\n'.format(output_idx, int(is_word)))
                output_idx += 1


generate_csv()

In [0]:
!kaggle competitions submit -c title-generation -m "BertSum split model_step_88000.pt" -f submission.csv 

100% 2.84M/2.84M [00:11<00:00, 270kB/s]
Successfully submitted to Arxiv Title Generation

In [0]:
#save checkpoint
#!cp /content/BertSum/logs/model/model_step_10000.pt /content/drive/My\ Drive/nlp/nlp_model/arxiv_bert

In [0]:
# !apt-get install libmagic-dev
# !pip install python-magic
# import magic
# mime = magic.Magic(mime=True)
# mime.from_file('/content/drive/My Drive/step60000.candidate')

# with open('/content/drive/My Drive/step130000.candidate') as myfile:
#     for i in range(10):
#         print( next(myfile))